# functions

In [1]:
# basic libraries
from pathlib import Path

# maths and tiff
import numpy as np
import tifffile
from PIL import Image

# jupyter widgets
from IPython.display import display

#plotting
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import FileLink

import io


In [3]:
def split_tiff_volume(volume):
    # Calculate the index of the middle row
    mid_height = volume.shape[0] // 2
    
    # Split the volume into two halves
    top_half = volume[:mid_height]
    bottom_half = volume[mid_height:]
    
    return top_half, bottom_half


def load_tiff_volume(input_folder):
    # List all TIFF files in the directory
    tiff_files = sorted(input_folder.glob("*.tif"))
    
    # Read the first file to get the shape and dtype
    with tifffile.TiffFile(tiff_files[0]) as tif:
        sample = tif.asarray()
        if len(sample.shape) == 2:
            height, width = sample.shape
            channels = 1
        else:
            height, width, channels = sample.shape

    num_files = len(tiff_files)

    # Create an empty volume array
    volume = np.empty((height, width, num_files, channels), dtype=sample.dtype)

    # Load each TIFF file into the volume array
    for i, file in enumerate(tiff_files):
        with tifffile.TiffFile(file) as tif:
            image_data = tif.asarray()
            # If the image data is 2D, we add an extra dimension to make it 3D
            if len(image_data.shape) == 2:
                image_data = image_data[..., np.newaxis]
            volume[:, :, i, :] = image_data
    return volume

In [4]:
def differential_analysis(volume):
    # Logic to calculate the density difference between adjacent slices
    num_slices = volume.shape[2]

    # Create an empty volume array for the density differences
    # The number of slices is one less than the original volume
    volume_diff = np.empty((volume.shape[0], volume.shape[1], num_slices - 1), dtype=volume.dtype)

    # Calculate the change in density for each pair of slices
    for i in range(1, num_slices):
        # Get the current and previous slice
        current_slice = np.squeeze(volume[:, :, i])
        prev_slice = np.squeeze(volume[:, :, i - 1])

        # Calculate the change in density from the previous slice
        delta_density = current_slice - prev_slice

        # Add the absolute value of the change in density to the volume_diff
        volume_diff[:, :, i - 1] = np.abs(delta_density)
        
    return volume_diff

def calculate_average_density(volume, slice_range=None):
    # Logic to calculate average density over a range of slices
    # If no slice range is specified, use the entire volume
    if slice_range is None:
        slice_range = slice(0, volume.shape[2])

    # Select the slices in the specified range
    volume_slice = volume[:, :, slice_range]

    # Calculate the average density for each point over the Z axis
    average_density = np.mean(volume_slice, axis=2)

    return average_density

In [5]:
def apply_gradient(image, skew=1.0, gradient_type='linear'):
    """
    Apply a specified gradient to an image.

    Returns:
    numpy array: The image after applying the gradient.
    """
    # Normalize the image
    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))

    # Apply the specified gradient
    if gradient_type == 'linear':
        skewed_image = normalized_image ** skew
    elif gradient_type == 'quadratic':
        skewed_image = normalized_image ** (2 * skew)
    elif gradient_type == 'logarithmic':
        skewed_image = np.log1p(normalized_image) ** skew
    elif gradient_type == 'parabolic':
        skewed_image = (1 - (1 - normalized_image) ** 2) ** skew
    else:
        raise ValueError("Invalid gradient type. Options are 'linear', 'quadratic', 'logarithmic', 'parabolic'.")

    # Map the skewed image values to the range 0-255
    return np.interp(skewed_image, (np.min(skewed_image), np.max(skewed_image)), (0, 255))

In [6]:
def process_volume(volume, gradient_type='linear', opacity_threshold=0.0, skew=1.0):
    """
    Process a 3D volume using a specified gradient.
    
    Returns:
    numpy array: The processed 3D volume.
    """
    # Logic to process the volume using the apply_gradient function
    num_slices = volume.shape[2]
    processed_slices = []

    for i in range(num_slices - 1):
        # Get the slice image
        slice_image = volume[:, :, i]

        # Apply the opacity threshold
        slice_image[slice_image <= opacity_threshold] = 0.0

        # Apply the gradient function with the skew value
        processed_slice = apply_gradient(slice_image, skew=skew, gradient_type=gradient_type)

        # Append the processed slice to the list
        processed_slices.append(processed_slice)

    # Stack the processed slices back together to form a new volume
    processed_volume = np.dstack(processed_slices)
    return processed_volume



In [7]:
def plot_histogram(data1, data2, title1, title2, xlabel, ylabel, bufmode=False):
    # Logic to plot histogram
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
    
    # Plot slide sums histogram
    ax1.bar(range(len(data1)), data1, color='blue', alpha=0.7)
    ax1.set_title(title1)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(ylabel)
    ax1.grid(True)
    
    # Plot z-scores
    ax2.bar(range(len(data2)), data2, color='blue', alpha=0.7)
    ax2.axhline(0, color='red', linestyle='--', label='Mean Value')
    ax2.set_title(title2)
    ax2.set_xlabel(xlabel)
    ax2.set_ylabel('Z-Score')
    ax2.grid(True)
    
    if bufmode:
        # Save the plot to a BytesIO object
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)
        # Return the bytes
        return buf.getvalue()
    else:
        plt.tight_layout()
        plt.show()
    
def calculate_slide_data(volume):
    # Logic to calculate slide data for histogram
    slide_sums = np.sum(volume, axis=(0, 1))
    z_scores = (slide_sums - np.mean(slide_sums)) / np.std(slide_sums)
    return slide_sums, z_scores

In [8]:
def on_button1_clicked(b):
    # Here is where we use the widget values for processing
    global skew_value
    skew_value = skew_value_widget.value
    global opacity
    opacity = opacity_widget.value
    global gradient_type 
    gradient_type = gradient_type_widget.value
    # Process the volume and get the processed volume
    with status_widget:
        print('Updating Gradient')
    global volume
    global original_volume
    # perform the differential analysis
    volume = differential_analysis(original_volume)
    # apply gradient
    volume = process_volume(volume, gradient_type, opacity_threshold=opacity ,skew=skew_value)
    with status_widget:
        print('Gradient OK')
        print( gradient_type, opacity, skew_value)
        

    # Calculate histogram and z-scores for each slide in the volume
    slide_sums, z_scores = calculate_slide_data(volume)
    # Plot slide histogram and z-scores side by side
    #     plot_histogram(slide_sums, z_scores, 'Density per Slide', 'Z-Score of Density in Each Slide', 'Slide Number', 'Sum of Pixel Values')
    # feeding histogram
    histogram_widget.value = plot_histogram(slide_sums, z_scores, 
                                            'Density per Slide', 'Z-Score of Density in Each Slide', 
                                            'Slide Number', 'Sum of Pixel Values', bufmode=True)

    
def on_button2_clicked(b):
    # Here is where we use the widget value for prompting result
    global slice_range 
    slice_range = slice(slice_range_widget.value[0], slice_range_widget.value[1])
    avg_density = calculate_average_density(volume, slice_range)

    # Normalize the image array to 0-255
    normalized_image = (avg_density - np.min(avg_density)) / (np.max(avg_density) - np.min(avg_density))
    
    # Invert the image
    inverted_image = 1 - normalized_image
    
    # Convert the image array to uint8
    uint8_image = (inverted_image * 255).astype(np.uint8)
    
    # Create a PIL Image
    pil_image = Image.fromarray(uint8_image)
    
    # Create a byte buffer for the image
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    # Update the image widget
    image_widget.value = img_byte_arr.getvalue()


def on_download_button_clicked(b):
    # Save the image to a file
    image_path = "%s_%s_%s_%s_%s_%s%s.png" % (input_folder_name, slice_range.start, slice_range.stop, gradient_type , skew_value, opacity, ('_split' if should_split else ''))
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    display(FileLink(image_path, result_html_prefix="Click here to download: "))


In [9]:
# Widget for the 'skew_value' float variable
skew_value_widget = widgets.FloatSlider(
    value=10,
    min=-20,
    max=20,
    step=1,
    description='Skew',
    continuous_update=False
)

# Widget for the 'opacity' float variable
opacity_widget = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='Opacity',
    continuous_update=False
)

# Widget for the 'gradient_type' string variable
gradient_type_widget = widgets.Dropdown(
    options=['linear', 'quadratic', 'logarithmic', 'parabolic'],
    value='linear',
    description='Grad.Type',
)

# Widget for the 'slice_range' slice variable
slice_range_widget = widgets.IntRangeSlider(
    value=[29, 42],
    min=0,
    max=100,  # replace with the appropriate max value
    step=1,
    description='Range',
    continuous_update=False
)

# Button for updating volume processing
update_button1 = widgets.Button(
    description='Update Processing',
)

# Text widget for status
status_widget = widgets.Output(layout={'border': '1px solid black'})


update_button1.on_click(on_button1_clicked)

# Create the image widget for slice 
image_widget = widgets.Image(format='png', width=1000, height=1000)

# Create the image widget for histogram 
histogram_widget = widgets.Image(format='png', width=300, height=300)

# Button for updating result prompting
update_button2 = widgets.Button(description='Update Result')

update_button2.on_click(on_button2_clicked)


# Create the download button
download_button = widgets.Button(description='Download Image')

download_button.on_click(on_download_button_clicked)


# Create layout with padding for the options column
options_layout = widgets.Layout(margin='0px 0px 0px 0px', width= '50%')
options_column = widgets.VBox([ gradient_type_widget, skew_value_widget, opacity_widget, update_button1, histogram_widget,status_widget])

# Create layout with padding for the image column
image_layout = widgets.Layout(margin='0px 0px 0px 20px', width= '50%')
image_column = widgets.VBox([slice_range_widget, update_button2, download_button, image_widget], layout=image_layout)

# Arrange columns side by side
ui = widgets.HBox([options_column, image_column])

In [10]:
# Display widgets
display(ui)

# START

In [11]:
# Set the path to folder of TIFF files
input_folder_name = "output/20230512123446_22/Tile_1_1"

# Source: http://dl.ash2txt.org/full-scrolls/Scroll1.volpkg/paths/20230512123446/
# Credits: @Hari-Sheldon, Vesuvian Challenge project.

input_folder = Path(input_folder_name)

In [12]:
# Create a volume from the source material
data = load_tiff_volume(input_folder)

# Choose whether to split the volume or not
should_split = False  # Set this variable to determine if splitting should be done

if should_split:
    # Split the volume and keep only the bottom half
    _, original_volume = split_tiff_volume(data)
else:
    original_volume = data

# We can void the data variable to save memory for later
del data

In [13]:
# perform the differential analysis
volume = differential_analysis(original_volume)

# NOTE, from here on, the volume not the original one,
# this is a map of CHANGE in density, not the density itself   

# Specify the desired skew value, threshold and gradient method
skew_value = 10
opacity = 0.4
gradient_type = 'logarithmic'

# modes:'linear', 'quadratic', 'logarithmic', 'parabolic':

# Process the volume and get the processed volume
volume = process_volume(volume, gradient_type, opacity_threshold=opacity ,skew=skew_value)


In [14]:
# If  image is in float format
# if avg_density.dtype.kind == 'f':
#     avg_density = (255 * (avg_density - np.min(avg_density)) / np.ptp(avg_density)).astype(np.uint8)

# # Create an Image object from the NumPy array
# img = Image.fromarray(avg_density)

# # Save the image as a TIFF file
# img.save("%s_%s_%s_%s_%s_%s%s.tiff" % (input_folder_name, slice_range.start, slice_range.stop, gradient_type , skew_value, opacity, ('_split' if should_split else ''  ) ))


In [15]:
# Save the processed slices as TIFF files in the alpha_folder

# Create a folder to save the alpha slices with gradient channel
# afn= input_folder_name + '_alpha_slices'
# alpha_folder = Path(afn)
# alpha_folder.mkdir(parents=True, exist_ok=True)
# save_slices_as_tiff(volume, alpha_folder)